# G2Engine

In [ ]:
import com.senzing.g2.engine.G2Engine;
import com.senzing.g2.engine.G2JNI;
import com.senzing.g2.engine.Result;

## Initialization

To start using the Senzing engine, you must first create and initialize an instance of the engine. This should be done once per process.

Create a new instance of the Senzing engine and assign it to a variable. Then, call the appropriate initialization method (such as `initV2`) to initialize the engine.

The init and `initAndGetConfigID` functions are deprecated. The version-2 functions (such as `initV2`) should be used instead.

During the call, the initialization methods accept the following parameters:

- **moduleName:** A short name given to this instance of the G2 engine (i.e. your G2Module object)
- **iniParams:** A JSON document containing system parameters (see the section called "Initialization Parameters")
- **iniFilename:** A fully qualified path to the G2 engine INI file (often /opt/senzing/g2/python/G2Module.ini)
- **verboseLogging:** A boolean which enables diagnostic logging - this will print a massive amount of information to stdout (default = False)
- **configID:** (optional in version-1 functions) The identifier value for the engine configuration can be returned here. (Note: as C does not accept optional parameters, C users should call the `initAndGetConfigID()` function to initialize the Senzing engine and return the engine's configuration ID)
- **initConfigID:** (required in some version-2 functions) The identifier value for the engine configuration that should be used. This must be a configuration ID that has been registered in the system using the G2ConfigMgr API.

In [ ]:
String G2_MODULE_NAME = "EngineSample";
String iniParams = "{\"PIPELINE\": {\"CONFIGPATH\" : \"/etc/opt/senzing\", \"SUPPORTPATH\": \"/opt/senzing/data/1.0.0\", \"RESOURCEPATH\": \"/opt/senzing/g2/resources\"}, \"SQL\": {\"CONNECTION\": \"sqlite3://na:na@/opt/senzing/g2/sqldb/G2C.db\"}}";
boolean verboseLogging = true;
G2Engine engine = new G2JNI();

int ret = engine.initV2(G2_MODULE_NAME, iniParams, verboseLogging);
System.out.print(ret);
engine.getLastException();

## Prime Engine

The `primeEngine()` method may optionally be called to pre-initialize some of the heavier weight internal resources of the G2 engine.

In [ ]:
int ret = engine.primeEngine();
if(ret!=0)
    System.out.println(engine.getLastException());
System.out.print(ret)

In [ ]:
String stats = engine.stats();
System.out.println("STATS: " + stats);

## addRecord()

Once the Senzing engine is initialized, use addRecord() to load a record into the Senzing repository -- addRecord() can be called as many times as desired and from multiple threads at the same time. The addRecord() function returns "0" upon success, and accepts four parameters as input:

- **datasource_code:** The name of the data source the record is associated with. This value is configurable to the system
- **record_id:** The record ID, used to identify distinct records
- **data_string:** A JSON document with the attribute data for the record
- **load_id:** The observation load ID for the record; value can be null and will default to data_source


In [ ]:
String dataSourceCode = "TEST";
String recordID = "RECORD3";
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"JANE\", \"NAME_LAST\": \"SMITH\", \"ADDR_TYPE\": \"HOME\", \"ADDR_LINE1\": \"653 STATE ROUTE 7\", \"ADDR_CITY\": \"FRESNO\", \"ADDR_STATE\": \"CA\", \"ADDR_POSTAL_CODE\": \"55073-1234\"}";
String loadID = null;

/** addRecord */
int ret = engine.addRecord(dataSourceCode, recordID, jsonData, loadID);

/** addRecordWithReturnedRecordID */
StringBuffer recordID2 = new StringBuffer();
int ret2 = engine.addRecordWithReturnedRecordID(dataSourceCode, recordID2, jsonData, loadID);

System.out.println("New Record ID: " + recordID2.toString());

/** addRecordWithInfo */
StringBuffer responseBuffer = new StringBuffer();
int flags = 0;
int ret3 = engine.addRecordWithInfo(dataSourceCode, recordID, jsonData, loadID, flags, responseBuffer);

System.out.println("Modified Entities: " + responseBuffer.toString());

## getRecord()

Use `getRecord()` to retrieve a single record from the data repository; the record is assigned in JSON form to a user-designated buffer, and the function itself returns "0" upon success. Once the Senzing engine is initialized, `getRecord()` can be called as many times as desired and from multiple threads at the same time. The `getRecord()` function accepts the following parameters as input:

- **data_source:** The name of the data source the record is associated with. This value is configurable to the system
- **record_id:** The record ID, used to identify the record for retrieval
- **response:** A memory buffer for returning the response document; if an error occurred, an error response is stored here

In [ ]:
String dataSourceCode = "TEST";
String recordID = "RECORD3";
String loadID = null;

/** deleteRecord */
int ret = engine.deleteRecord(dataSourceCode, recordID, loadID);

/** deleteRecordWithInfo */
StringBuffer responseBuffer = new StringBuffer();
int flags = 0;
int ret2 = engine.deleteRecordWithInfo(dataSourceCode, recordID, loadID, flags, responseBuffer);
System.out.println("Modified Entities: " + responseBuffer.toString());

## getEntityByRecordID()

Use `getEntityByRecordID()` to retrieve entity data based on the record ID of a particular data record. This function accepts the following parameters as input:

- **datasource_code:** The name of the data source the record is associated with. This value is configurable to the system
- **record_id:** The record ID for a particular data record
- **response:** A memory buffer for returning the response document; if an error occurred, an error response is stored here.

In [ ]:
String dataSourceCode = "TEST";
String recordID = "1";

long entityID = 1;
StringBuffer response = new StringBuffer();

int ret2 = engine.getEntityByEntityID(entityID, response);
System.out.println("GET Entity: " + response.toString());

response = new StringBuffer();

int ret3 = engine.getEntityByRecordID(dataSourceCode,recordID,response);
System.out.println("GET Entity: " + response.toString());
if(ret2!=0)
    System.out.println(engine.getLastException());
if(ret3!=0)
    System.out.println(engine.getLastException());
System.out.println(ret2)

## searchByAttributes()

Use `searchByAttributes()` to retrieve entity data based on a user-specified set of entity attributes. This function accepts the following parameters as input:

- **data_string:** A JSON document with the attribute data to search for
- **response:** A memory buffer for returning the response document; if an error occurred, an error response is stored here.

In [ ]:
String dataSourceCode = "TEST";
String recordID = "1";

response = new StringBuffer();

int ret4 = engine.searchByAttributes(jsonData, response);
System.out.println("GET Entity: " + response.toString());
if(ret4!=0)
    System.out.println(engine.getLastException());
System.out.println(ret4)

# Replace the record
Use the `replaceRecord()` function to update or replace a record in the data repository (if record doesn't exist, a new record is added to the data repository. Like the above functions, `replaceRecord()` returns "0" upon success, and it can be called as many times as desired and from multiple threads at the same time. The `replaceRecord()` function accepts four parameters as input:

- **dataSourceCode:** The name of the data source the record is associated with. This value is configurable to the system
- **recordID:** The record ID, used to identify distinct records
- **jsonData:** A JSON document with the attribute data for the record
- **loadID:** The observation load ID for the record; value can be null and will default to dataSourceCode

In [ ]:
String dataSourceCode = "TEST";
String recordID = "1";
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"JANE\", \"NAME_LAST\": \"ADAMS\", \"ADDR_TYPE\": \"HOME\", \"ADDR_LINE1\": \"653 STATE ROUTE 7\", \"ADDR_CITY\": \"FRESNO\", \"ADDR_STATE\": \"CA\", \"ADDR_POSTAL_CODE\": \"55073-1234\"}";
String loadID = null;

int ret = engine.replaceRecord(dataSourceCode, recordID, jsonData, loadID);
if(ret!=0)
    System.out.println(engine.getLastException());
System.out.println(ret)

Do ```getRecord()``` again to see the changes

In [ ]:
String dataSourceCode = "TEST";
String recordID = "1";
StringBuffer response = new StringBuffer();

int ret = engine.getRecord(dataSourceCode, recordID, response);

System.out.println("GET RECORD: " + response.toString());
System.out.println("");
if(ret!=0)
    System.out.println(engine.getLastException());
System.out.print(ret)

## Export JSON Entity Report

There are three steps to exporting resolved entity data from the G2Engine object in JSON format. First, use the `exportJSONEntityReport()` method to generate a long integer, referred to here as an 'exportHandle'. The `exportJSONEntityReport()` method accepts one parameter as input:

- **flags**: An integer specifying which entity details should be included in the export. See the "Entity Export Flags" section for further details.

Second, use the fetchNext() method to read the exportHandle and export a row of JSON output containing the entity data for a single entity. Note that successive calls of fetchNext() will export successive rows of entity data. The fetchNext() method accepts the following parameters as input:

- **exportHandle:** A long integer from which resolved entity data may be read and exported
- **response:** A memory buffer for returning the response document; if an error occurred, an error response is stored here.

In [ ]:
int flags = engine.G2_EXPORT_INCLUDE_ALL_ENTITIES;

long exportHandle = engine.exportJSONEntityReport(flags);

String response = engine.fetchNext(exportHandle);
System.out.println(response);

engine.closeExport(exportHandle);

## Export CSV Entity Report

There are three steps to exporting resolved entity data from the G2Engine object in CSV format. First, use the `exportCSVEntityReport()` method to generate a long integer, referred to here as an 'exportHandle'. The `exportCSVEntityReport()` method accepts one parameter as input:

- **flags:** An integer specifying which entity details should be included in the export. See the "Entity Export Flags" section for further details.
Second, use the `fetchNext()` method to read the exportHandle and export a row of CSV output containing the entity data for a single entity. Note that the first call of `fetchNext()` may yield a header row, and that successive calls of `fetchNext()` will export successive rows of entity data. The `fetchNext()` method accepts the following parameters as input:

- **exportHandle:** A long integer from which resolved entity data may be read and exported
- **response:** A memory buffer for returning the response document; if an error occurred, an error response is stored here

In [ ]:
int flags = engine.G2_EXPORT_INCLUDE_ALL_ENTITIES;

long exportHandle = engine.exportCSVEntityReport(flags);

String response = engine.fetchNext(exportHandle);
System.out.println(response);

engine.closeExport(exportHandle);

## Finding Paths
The `FindPathByEntityID()` and `FindPathByRecordID()` functions can be used to find single relationship paths between two entities. Paths are found using known relationships with other entities.

Entities can be searched for by either Entity ID or by Record ID, depending on which function is chosen.

These functions have the following parameters:

- **entityID1:** The entity ID for the starting entity of the search path
- **entityID2:** The entity ID for the ending entity of the search path
- **dataSourceCode1:** The data source for the starting entity of the search path
- **recordID1:** The record ID for the starting entity of the search path
- **dataSourceCode2:** The data source for the ending entity of the search path
- **recordID2:** The record ID for the ending entity of the search path
- **maxDegree:** The number of relationship degrees to search

First you will need to create some records so that you have some that you can compare. Can you see what is the same between this record and the previous one?

In [ ]:
String dataSourceCode = "TEST";
String recordID = "2";
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"JOHN\", \"NAME_LAST\": \"SMITH\", \"ADDR_TYPE\": \"HOME\", \"ADDR_LINE1\": \"753 STATE ROUTE 8\", \"ADDR_CITY\": \"FRESNO\", \"ADDR_STATE\": \"CA\", \"ADDR_POSTAL_CODE\": \"55073-1234\"}";
String loadID = null;

int ret = engine.addRecord(dataSourceCode, recordID, jsonData, loadID);

StringBuffer recordID2 = new StringBuffer();
int ret2 = engine.addRecordWithReturnedRecordID(dataSourceCode, recordID2, jsonData, loadID);

System.out.println("New Record ID: " + recordID2.toString());
if(ret!=0)
    System.out.println(engine.getLastException());
if(ret2!=0)
    System.out.println(engine.getLastException());
System.out.println(ret)

## `FindPathByEntityID()`

In [ ]:
long entityID1 = 1;
long entityID2 = 2;
int maxDegree = 3;

StringBuffer response = new StringBuffer();

int ret_code = engine.findPathByEntityID(entityID1,entityID2,maxDegree,response);

System.out.println("Path result document: " + response.toString());
if(ret_code!=0)
    System.out.println(engine.getLastException());
System.out.print(ret_code)

## `FindPathByRecordID()`

In [ ]:
String dataSourceCode1 = new String("TEST");
String recordID1 = new String("1");
String dataSourceCode2 = new String("TEST");
String recordID2 = new String("2");

ret_code = engine.findPathByRecordID(dataSourceCode1,recordID1,dataSourceCode2,recordID2,maxDegree,response);

System.out.println("Path result document: " + response.toString());
if(ret_code!=0)
    System.out.println(engine.getLastException());
System.out.println(ret_code)

## Finding Paths with Exclusions
The `FindPathExcludingByEntityID()` and `FindPathExcludingByRecordID()` functions can be used to find single relationship paths between two entities. Paths are found using known relationships with other entities. In addition, it will find paths that exclude certain entities from being on the path.

Entities can be searched for by either Entity ID or by Record ID, depending on which function is chosen. Additionally, entities to be excluded can also be specified by either Entity ID or by Record ID.

When excluding entities, the user may choose to either (a) strictly exclude the entities, or (b) prefer to exclude the entities, but still include them if no other path is found. By default, entities will be strictly excluded. A "preferred exclude" may be done by specifying the G2_FIND_PATH_PREFER_EXCLUDE control flag.

These functions have the following parameters:

- **entityID1:** The entity ID for the starting entity of the search path
- **entityID2:** The entity ID for the ending entity of the search path
- **dataSourceCode1:** The data source for the starting entity of the search path
- **recordID1:** The record ID for the starting entity of the search path
- **dataSourceCode2:** The data source for the ending entity of the search path
- **recordID2:** The record ID for the ending entity of the search path
- **maxDegree:** The number of relationship degrees to search
- **excludedEntities:** Entities that should be avoided on the path (JSON document)
- **flags:** Operational flags

In [ ]:
String dataSourceCode = "TEST";
String recordID = "3";
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"SAM\", \"NAME_LAST\": \"MILLER\", \"ADDR_TYPE\": \"HOME\", \"ADDR_LINE1\": \"753 STATE ROUTE 8\", \"ADDR_CITY\": \"FRESNO\", \"ADDR_STATE\": \"CA\", \"ADDR_POSTAL_CODE\": \"55073-1234\", \"SSN_NUMBER\": \"111-11-1111\"}";
String loadID = null;

int ret = engine.addRecord(dataSourceCode, recordID, jsonData, loadID);

StringBuffer recordID2 = new StringBuffer();
int ret2 = engine.addRecordWithReturnedRecordID(dataSourceCode, recordID2, jsonData, loadID);

System.out.println("New Record ID: " + recordID2.toString());
if(ret!=0)
    System.out.println(engine.getLastException());
if(ret2!=0)
    System.out.println(engine.getLastException());
System.out.println(ret)

## `FindPathExcludingByEntityID()`

In [ ]:
long entityID1 = 2;
long entityID2 = 3;
int maxDegree = 4;
String excludedEntities = new String("{\"ENTITIES\":[{\"ENTITY_ID\":\"1\"}]}");
int flags = G2Engine.G2_EXPORT_DEFAULT_FLAGS;

StringBuffer response = new StringBuffer();

int ret_code = engine.findPathExcludingByEntityID(entityID1,entityID2,maxDegree,excludedEntities,flags,response);
System.out.println("Path result document: " + response.toString());
if(ret_code!=0)
    System.out.println(engine.getLastException());
System.out.println(ret_code);

## `FindPathExcludingByRecordID()`

In [ ]:
String dataSourceCode1 = new String("TEST");
String recordID1 = new String("2");
String dataSourceCode2 = new String("TEST");
String recordID2 = new String("3");
String excludedRecords = new String("{\"RECORDS\":[{\"RECORD_ID\":\"1\",\"DATA_SOURCE\":\"TEST\"}]}");

ret_code = engine.findPathExcludingByRecordID(dataSourceCode1,recordID1,dataSourceCode2,recordID2,maxDegree,excludedRecords,flags,response);

System.out.println("Path result document: " + response.toString());
if(ret_code!=0)
    System.out.println(engine.getLastException());
System.out.println(ret_code);

## Finding Paths with Required Sources
The `FindPathIncludingSourceByEntityID()` and `FindPathIncludingSourceByRecordID()` functions can be used to find single relationship paths between two entities. In addition, one of the enties along the path must include a specified data source.

Entities can be searched for by either Entity ID or by Record ID, depending on which function is chosen. The required data source or sources are specified by a json document list.

Specific entities may also be excluded, using the same methodology as the `FindPathExcludingByEntityID()` and `FindPathExcludingByRecordID()` functions use.

These functions have the following parameters:

- **entityID1:** The entity ID for the starting entity of the search path
- **entityID2:** The entity ID for the ending entity of the search path
- **dataSourceCode1:** The data source for the starting entity of the search path
- **recordID1:** The record ID for the starting entity of the search path
- **dataSourceCode2:** The data source for the ending entity of the search path
- **recordID2:** The record ID for the ending entity of the search path
- **maxDegree:** The number of relationship degrees to search
- **excludedEntities:** Entities that should be avoided on the path (JSON document)
- **requiredDsrcs:** Entities that should be avoided on the path (JSON document)
- **flags:** Operational flags

In [ ]:
long entityID1 = 2;
long entityID2 = 1;
int maxDegree = 4;
String excludedEntities = new String("{\"ENTITIES\":[{\"ENTITY_ID\":\"1\"}]}");
String requiredDsrcs = new String("{\"DATA_SOURCES\":[\"TEST\"]}");
int flags = 0;

StringBuffer response = new StringBuffer();

int ret_code = engine.findPathIncludingSourceByEntityID(entityID1,entityID2,maxDegree,excludedEntities,requiredDsrcs,flags,response);

System.out.println("Path result document: " + response.toString());

String dataSourceCode1 = new String("TEST");
String recordID1 = new String("2");
String dataSourceCode2 = new String("TEST");
String recordID2 = new String("3");
String excludedRecords = new String("{\"RECORDS\":[{\"RECORD_ID\":\"1\",\"DATA_SOURCE\":\"TEST\"}]}");

ret_code = engine.findPathIncludingSourceByRecordID(dataSourceCode1,recordID1,dataSourceCode2,recordID2,maxDegree,excludedRecords,requiredDsrcs,flags,response);

System.out.println("Path result document: " + response.toString());
if(ret_code!=0)
    System.out.println(engine.getLastException());
System.out.print(ret_code);

## Redo Processing
Redo records are automatically created by Senzing when certain conditions occur where it believes more processing may be needed.  Some examples:
* A value becomes generic and previous decisions may need to be revisited
* Clean up after some record deletes
* Detected related entities were being changed at the same time
* A table inconsistency exists, potentially after a non-graceful shutdown
First we will need to have a total of 6 data sources so let's add 4 more

In [ ]:
String dataSourceCode = "TEST";
String recordID = "4";
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"JANE\", \"NAME_LAST\": \"ADAMS\", \"SSN_NUMBER\": \"111-11-1111\"}";
String loadID = null;
int ret = engine.addRecord(dataSourceCode, recordID, jsonData, loadID);
if(ret!=0)
    System.out.println(engine.getLastException());
System.out.println(ret);

String dataSourceCode = "TEST";
String recordID = "5";
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"LILY\", \"NAME_LAST\": \"OWENS\", \"SSN_NUMBER\": \"111-11-1111\"}";
String loadID = null;
int ret = engine.addRecord(dataSourceCode, recordID, jsonData, loadID);
if(ret!=0)
    System.out.println(engine.getLastException());
System.out.println(ret);

String dataSourceCode = "TEST";
String recordID = "6";
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"AUGUST\", \"NAME_LAST\": \"Bauler\", \"SSN_NUMBER\": \"111-11-1111\"}";
String loadID = null;
int ret = engine.addRecord(dataSourceCode, recordID, jsonData, loadID);
if(ret!=0)
    System.out.println(engine.getLastException());
System.out.println(ret);

String dataSourceCode = "TEST";
String recordID = "7";
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"JACK\", \"NAME_LAST\": \"MILLER\", \"SSN_NUMBER\": \"111-11-1111\"}";
String loadID = null;
int ret = engine.addRecord(dataSourceCode, recordID, jsonData, loadID);
if(ret!=0)
    System.out.println(engine.getLastException());
System.out.println(ret);

String dataSourceCode = "TEST";
String recordID = "8";
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"LOGAN\", \"NAME_LAST\": \"WILLIAMS\", \"SSN_NUMBER\": \"111-11-1111\"}";
String loadID = null;
int ret = engine.addRecord(dataSourceCode, recordID, jsonData, loadID);
if(ret!=0)
    System.out.println(engine.getLastException());
System.out.println(ret);

## Counting the number of redos
This returns the number of redos within the processed records that are awaiting processing.

In [ ]:
long response = engine.countRedoRecords();
if(response<0)
    System.out.println(engine.getLastException());
System.out.print(response);

## Geting a redo record
Gets a redo record so that it can be processed

In [ ]:
StringBuffer response_string = new StringBuffer();
int response = engine.getRedoRecord(response_string);
if(response==0 && response_string.length()>0)
    response = engine.process(response_string.toString());
if(response!=0)
    System.out.println(engine.getLastException());
System.out.print(response)

In [ ]:
StringBuffer response_string = new StringBuffer();
int response = engine.getRedoRecord(response_string);
System.out.println(response_string);
System.out.println(response);

# Why records belong to an entity
The `WhyEntityByEntityID()` and `WhyEntityByRecordID()` functions can be used to determine why records belong to their resolved entities. These functions will compare the record data within an entity against the rest of the entity data, and show why they are connected. This is calculated based on the features that record data represents.

Records can be chosen by either Record ID or by Entity ID, depending on which function is chosen. If a single record ID is used, then comparison results for that single record will be generated, as part of its entity. If an Entity ID is used, then comparison results will be generated for every record within that entity.

These functions have the following parameters:

- **entityID:** The entity ID for the entity to be analyzed
- **dataSourceCode:** The data source for the record to be analyzed
- **recordID:** The record ID for the record to be analyzed
- **flags:** Control flags for outputting entities
They also have various arguments used to return response documents.

The functions return a JSON document that gives the results of the record analysis. The document contains a section called "WHY_RESULTS", which shows how specific records relate to the rest of the entity. It has a "WHY_KEY", which is similar to a match key, in defining the relevant connected data. It shows candidate keys for features that initially cause the records to be analyzed for a relationship, plus a series of feature scores that show how similar the feature data was.

The response document also contains a separate ENTITIES section, with the full information about the resolved entity. (Note: When working with this entity data, Senzing recommends using the flags G2_ENTITY_SHOW_FEATURES_EXPRESSED and G2_ENTITY_SHOW_FEATURES_STATS. This will provide detailed feature data that is not included by default, but is useful for understanding the WHY_RESULTS data.)

In [ ]:
/** define input variables */
long entityID = 1;

/** buffer response variables */
StringBuffer response = new StringBuffer();

/** find the why-information */
int ret_code = engine.whyEntityByEntityIDV2(entityID,flags,response);

/** print the results */
System.out.println("Result document: " + response.toString()+"\n");

/** define input variables */
String dataSourceCode = new String("PEOPLE");
String recordID = new String("1001");

/** find the why-information */
ret_code = engine.whyEntityByRecordIDV2(dataSourceCode,recordID,flags,response);

/** print the results */
System.out.println("Result document: " + response.toString());

## Deleting Records
use `deleteRecord()` to remove a record from the data repository (returns "0" upon success) ; `deleteRecord()` can be called as many times as desired and from multiple threads at the same time. The `deleteRecord()` function accepts three parameters as input:

- **dataSourceCode:** The name of the data source the record is associated with. This value is configurable to the system
- **recordID:** The record ID, used to identify distinct records
- **loadID:** The observation load ID for the record; value can be null and will default to dataSourceCode

In [ ]:
ret = engine.deleteRecord(dataSourceCode,recordID, loadID);
if(ret!=0)
    System.out.println(engine.getLastException());
System.out.print(ret)

Attempt to get the record again. It should error and give an output similar to "Unknown record" and a return value of -2.

In [ ]:
String dataSourceCode = "TEST";
String recordID = "1";
StringBuffer response = new StringBuffer();
int ret = engine.getRecord(dataSourceCode, recordID, response);

if(ret!=0)
    System.out.println(engine.getLastException());
System.out.println(ret)

## Purge Repository
To purge the G2 repository, use the aptly named `purgeRepository()` method. This will remove every record in your current repository.

In [ ]:
int ret = engine.purgeRepository();
if(ret!=0)
    System.out.println(engine.getLastException());
System.out.print(ret)